In [1]:
from sklearn.ensemble import GradientBoostingRegressor
X = [[0,0],[1,1],[3,2],[4,5]]
x = [[2,1],[1,3]]
y = [1,2,3,4]
gbdt = GradientBoostingRegressor(n_estimators = 3)
gbdt_model = gbdt.fit(X,y)
print(gbdt_model.feature_importances_)

[0.2939378 0.7060622]


In [2]:
for i in gbdt_model.estimators_:
    print(i[0].feature_importances_)

[0.2 0.8]
[0. 1.]
[0.8 0.2]


In [3]:
#先导入我们需要的库
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split#用于划分测试集与训练集

from sklearn.datasets import load_iris #使用sklearn库中的iris数据集
data = load_iris()
X = data.data #特征
Y=data.target #类标

X = X[0:100,:]
Y=Y[0:100] #我们做个二分类，iris本来是三个类，前100 个包括两个类
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,\
                                               test_size=0.2,random_state=0)#数据划分

clas = GradientBoostingClassifier(random_state=2020)#我们用的默认参数，如果数据比较复杂，需要调参
clas.fit(X_train,Y_train)#训练模型
clas.predict(X_train)#预测训练集
clas.predict(X_test)#预测测试集
print("训练集准确率：%s"%clas.score(X_train,Y_train)) #输出测试集准确度
print("测试集准确率：%s"%clas.score(X_test,Y_test))   #输出测试集准确度

训练集准确率：1.0
测试集准确率：1.0


In [12]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier, GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,auc
# from sklearn.externals import joblib
# from sklearn.externals.six import StringIO
from sklearn import tree
import pydotplus
import joblib
from six import StringIO

n_estimator = 4 # the number of base trees
X, y = make_classification(n_samples=100,n_features=4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)
X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train,y_train,test_size=0.3)

grd = GradientBoostingClassifier(n_estimators=n_estimator,max_depth=3)
grd.fit(X_train, y_train)
score_feature = grd.feature_importances_
print(score_feature)
joblib.dump(grd,'gbdt_model/grd_model.m',compress = 3)

m1 = grd.apply(X_train)[:, :, 0]
# print (m1)
grd_enc = OneHotEncoder()
grd_lm = LogisticRegression()
grd_enc.fit(m1)
joblib.dump(grd_enc,'gbdt_model/grd_enc_model.m',compress = 3)

grd_lm.fit(grd_enc.transform(grd.apply(X_train_lr)[:, :, 0]), y_train_lr)
joblib.dump(grd_lm,'gbdt_model/grd_lm_model.m',compress = 3)

# save the training model

m2 = grd_enc.transform(grd.apply(X_train_lr)[:, :, 0])

print (m2.toarray())

# grd = joblib.load('gbdt_model/grd_model.m')
# grd_enc = joblib.load('gbdt_model/grd_enc_model.m')
# grd_lm = joblib.load('gbdt_model/grd_lm_model.m')


y_pred_grd_lm = grd_lm.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0]))
print(grd_lm.coef_)
#print(y_pred_grd_lm)

acc = grd_lm.score(grd_enc.transform(grd.apply(X_test)[:, :, 0]),y_test)
print("acc is ",acc)

fpr_grd_lm, tpr_grd_lm, _ = roc_curve(y_test, y_pred_grd_lm[:,1])
roc_auc = auc(fpr_grd_lm,tpr_grd_lm)
print("auc is ",roc_auc)

dot_data = StringIO()
tree.export_graphviz(grd.estimators_[0,0],out_file = dot_data,node_ids=True,filled=True,rounded=True,special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("yang.pdf")
print('Visible tree plot saved as pdf.')


[0.18775906 0.64646465 0.         0.1657763 ]


FileNotFoundError: [Errno 2] No such file or directory: 'gbdt_model/grd_model.m'

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
# from sklearn.externals import joblib
import numpy as np
import joblib

# 以分隔符,读取文件，得到的是一个二维列表
iris = np.loadtxt('iris.data', dtype=str, delimiter=',', unpack=False, encoding='utf-8')

# 前4列是特征
data = iris[:, :4].astype(np.float)
# 最后一列是标签，我们将其转换为二维列表
target = iris[:, -1][:, np.newaxis]

# 对标签进行onehot编码后还原成数字
enc = OneHotEncoder()
target = enc.fit_transform(target).astype(np.int).toarray()
target = [list(oh).index(1) for oh in target]

# 划分训练数据和测试数据
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=1)

# 模型训练
gbdt = GradientBoostingClassifier(n_estimators=3000, max_depth=2, min_samples_split=2, learning_rate=0.1)
gbdt.fit(X_train, y_train)

# 模型存储
joblib.dump(gbdt, 'gbdt_model.pkl')
# 模型加载
gbdt = joblib.load('gbdt_model.pkl')

# 模型预测
y_pred = gbdt.predict(X_test)

# 模型评估
print('The accuracy of prediction is:', accuracy_score(y_test, y_pred))

# 特征重要度
print('Feature importances:', list(gbdt.feature_importances_))

OSError: iris.data not found.

In [1]:
from sklearn import datasets
 
'''清空sklearn环境下所有数据'''
datasets.clear_data_home()
 
'''载入波士顿房价数据'''
 
X,y = datasets.load_boston(return_X_y=True)
 
'''获取自变量数据的形状'''
 
print(X.shape)
 
'''获取因变量数据的形状'''
 
print(y.shape)


(506, 13)
(506,)


In [5]:
print(type(X))

<class 'numpy.ndarray'>
